<a href="https://colab.research.google.com/github/percivalyan/SOUND-WAVEFORM/blob/main/SOUND_WAVEFORM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install librosa matplotlib numpy pydub

In [ ]:
from google.colab import files
from pydub import AudioSegment
import io

# Upload file
uploaded = files.upload()

# Convert uploaded file to WAV format if necessary
filename = list(uploaded.keys())[0]
file_extension = filename.split('.')[-1].lower()

# Load audio file with pydub
audio = AudioSegment.from_file(io.BytesIO(uploaded[filename]), format=file_extension)

# Check duration and trim if necessary
max_duration = 10 * 60 * 1000  # 10 minutes in milliseconds
if len(audio) > max_duration:
    audio = audio[:max_duration]

# Save to a temporary WAV file for further processing
wav_filename = '/tmp/temp_audio.wav'
audio.export(wav_filename, format='wav')


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load audio file
y, sr = librosa.load(wav_filename)

# Generate the spectrogram
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

# Plot the spectrogram
plt.figure(figsize=(10, 6))
librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.show()

# Plot the waveform
plt.figure(figsize=(10, 4))
plt.plot(np.linspace(0, len(y)/sr, num=len(y)), y)
plt.title('Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()

# Analyze pitch (frequency)
pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
pitch_freqs = []
for t in range(pitches.shape[1]):
    index = magnitudes[:, t].argmax()
    pitch_freqs.append(pitches[index, t])

# Plot pitch frequency over time
plt.figure(figsize=(10, 4))
plt.plot(np.linspace(0, len(y)/sr, num=len(pitch_freqs)), pitch_freqs, color='r')
plt.title('Pitch Frequency')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()


In [ ]:
def analyze_audio_description(y, sr):
    # Spectrogram Analysis
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    avg_dB = np.mean(D)
    max_dB = np.max(D)
    min_dB = np.min(D)
    freq_range = librosa.fft_frequencies(sr=sr)

    # Waveform Analysis
    duration = len(y) / sr
    max_amplitude = np.max(y)
    min_amplitude = np.min(y)

    # Pitch Frequency Analysis
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_freqs = []
    for t in range(pitches.shape[1]):
        index = magnitudes[:, t].argmax()
        pitch_freqs.append(pitches[index, t])

    # Calculating pitch statistics
    avg_pitch = np.mean(pitch_freqs)
    max_pitch = np.max(pitch_freqs)
    min_pitch = np.min(pitch_freqs)

    # Generating descriptive conclusions
    conclusions = {
        "Spectrogram Analysis": {
            "Average dB Level": f"{avg_dB:.2f} dB",
            "Maximum dB Level": f"{max_dB:.2f} dB",
            "Minimum dB Level": f"{min_dB:.2f} dB",
            "Frequency Range": f"{freq_range[0]:.2f} Hz to {freq_range[-1]:.2f} Hz"
        },
        "Waveform Analysis": {
            "Duration": f"{duration:.2f} seconds",
            "Maximum Amplitude": f"{max_amplitude:.2f}",
            "Minimum Amplitude": f"{min_amplitude:.2f}"
        },
        "Pitch Frequency Analysis": {
            "Average Pitch Frequency": f"{avg_pitch:.2f} Hz",
            "Maximum Pitch Frequency": f"{max_pitch:.2f} Hz",
            "Minimum Pitch Frequency": f"{min_pitch:.2f} Hz"
        }
    }

    # Print conclusions
    print("Descriptive Analysis of Audio Data:")
    for section, stats in conclusions.items():
        print(f"\n{section}:")
        for key, value in stats.items():
            print(f"  - {key}: {value}")

# Call the function to print the descriptive analysis
analyze_audio_description(y, sr)